In [46]:
import os

In [47]:
os.chdir("C:\\Users\\Lenovo-Pc\\End-to-End-Machine-Learning-Project\\research")


In [48]:
os.chdir("../")

In [52]:
%pwd

'C:\\Users\\Lenovo-Pc\\End-to-End-Machine-Learning-Project'

In [49]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [54]:
%pwd

'C:\\Users\\Lenovo-Pc\\End-to-End-Machine-Learning-Project'

In [64]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [65]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [69]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size




In [70]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [72]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-30 17:34:48,719: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-12-30 17:34:48,733: INFO: common: yaml file: params.yaml loaded successfully]
[2024-12-30 17:34:48,737: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-12-30 17:34:48,739: INFO: common: created directory at: artifacts]
[2024-12-30 17:34:48,741: INFO: common: created directory at: artifacts/data_ingestion]
[2024-12-30 17:34:49,806: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Server: GitHub.com
Date: Mon, 30 Dec 2024 22:34:49 GMT
Content-Type: text/html; charset=utf-8
Vary: X-PJAX, X-PJAX-Container, Turbo-Visit, Turbo-Frame, Accept-Encoding, Accept, X-Requested-With
ETag: W/"2b932d223447c2119e651b411671e859"
Cache-Control: max-age=0, private, must-revalidate
Strict-Transport-Security: max-age=31536000; includeSubdomains; preload
X-Frame-Options: deny
X-Content-Type-Options: nosniff
X-XSS-Protection: 0
Referrer-Policy: no-referrer-when-do

BadZipFile: File is not a zip file